In [ ]:
#@title Load utils
REGISTRY = (
    {
        "id": "SOURCE",
        "name": "oobabooga",
    },
    {
        "id": "NAME",
        "name": "text-generation-webui",
    },
)

for opt in REGISTRY:
    globals()[opt["id"]] = opt["name"]

BASE = f"https://github.com/{globals()['SOURCE']}/{globals()['NAME']}"
BASE_PATH = f"/content/{globals()['NAME']}"
NGROK = f"/content/{globals()['NAME']}/extensions/ngrok/requirements.txt"


In [ ]:
#@title Prevent disconnections
%%html
<audio src="https://oobabooga.github.io/silence.m4a" controls>

In [ ]:
#@title Download & install backend
!apt-get -y install -qq aria2
#!pip install flask-cloudflared
!pip install requests-html

!git clone $BASE
%cd $BASE_PATH
!pip install -r requirements.txt
!pip install -r $NGROK

In [ ]:
#@title Add model

%cd /content/

from requests_html import HTMLSession

OPTIONS = {
    1: {
        "id": "TheBloke/Echidna-13B-v0.3-GPTQ", 
        "file_list": []
    },
    2: {
        "id": "Sao10K/Fimbulvetr-11B-v2-GGUF", 
        "file_list": [
            "https://huggingface.co/Sao10K/Fimbulvetr-11B-v2-GGUF/blob/main/Fimbulvetr-11B-v2.q4_K_S.gguf"
        ]
    }
}

#@markdown ### Model presets
Preset = "2" #@param [1,2]

OPTION = OPTIONS[int(Preset)]
MODEL_ID = OPTION["id"]
MODEL_FILES = OPTION["file_list"]

#Get model name
MODEL_NAME = MODEL_ID.split(MODEL_ID[MODEL_ID.rfind("/")])[-1]

#@markdown ### Select model type
MODEL_TYPE = "gguf" #@param ["gptq", "gguf"]

#@markdown ### model file name based on backend and model type (empty by default)
renamed_file = "" #@param {type:"string"}

if MODEL_TYPE == "gptq":
    renamed_file = "gptq_model-4bit-128g.safetensors" 

if MODEL_TYPE == "gguf":
    file = list(filter(lambda filename: filename.endswith(".gguf"), MODEL_FILES))[0]
    renamed_file = file.split(file[file.rfind("/")])[-1]

#Get LINKS for GPTQ model
if MODEL_TYPE == "gptq":
    #Target repo
    REPO = f"https://huggingface.co/{MODEL_ID}/tree/main?not-for-all-audiences=true"

    #Get file urls
    session = HTMLSession()
    result = session.get(REPO)
    links = result.html.absolute_links

    #Create LINKS list
    FILE_TARGETS = (".json", ".model", ".safetensors", ".py")
    if len(MODEL_FILES) > 0:
        LINKS = MODEL_FILES
        for i in range(len(LINKS)):
            LINKS[i] = LINKS[i].replace("/blob/", "/resolve/")
    else:
        LINKS = []
        for link in links:     
            if any(target in link for target in FILE_TARGETS) and "/resolve/" in link: LINKS.append(link)

#Get LINKS for GGUF model (MUST HAVE urls in 'file_list'!)
if MODEL_TYPE == "gguf":
    LINKS = MODEL_FILES
    for i in range(len(LINKS)):
        LINKS[i] = LINKS[i].replace("/blob/", "/resolve/")

#Input file for aria2
shopping_list = "down_list.txt"

#Create file for aria2 using LINKS list
with open(shopping_list, 'a') as file:
    if MODEL_TYPE == "gguf":
        for url in LINKS:
            if ".gguf" in url: file.write(f"{url}\n out={renamed_file}\n")
    if MODEL_TYPE == "gptq":
        for url in LINKS:
            if ".safetensors" in url: file.write(f"{url}\n out={renamed_file}\n")
            elif ".model" in url: file.write(f"{url}\n out=tokenizer.model\n")
            else: file.write(f"{url}\n")

#Set models destination for backend
models_folder = f"{BASE_PATH}/models"

#Download all model config files
print(f"Downloading {MODEL_NAME}...\n")

if MODEL_TYPE == "gguf":
    !cd $models_folder
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M --summary-interval=5 --input-file=$shopping_list -d $models_folder
    !rm $shopping_list

if MODEL_TYPE == "gptq":
    !cd $models_folder && mkdir $MODEL_NAME
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M --summary-interval=5 --input-file=$shopping_list -d $models_folder/$MODEL_NAME
    !rm $shopping_list

In [ ]:
#@title LAUNCH! (Run again in case of cloudflare error)

%cd $BASE_PATH

provider = "cloudflare" #@param ["ngrok", "cloudflare"]
context = "4096" #@param {type:"string"}
gpu_layers = "60" #@param {type:"string"}

CONTEXT = int(context)
GPU_LAYERS = int(gpu_layers)

if provider == "ngrok":
    PROVIDER = "--extension ngrok"
else:
    PROVIDER = ""

if MODEL_TYPE == "gguf":
    LOADER = "LLAMACPP"
    MODEL_NAME = renamed_file
    !python server.py --share --nowebui --api --public-api $PROVIDER --model $MODEL_NAME --loader $LOADER --max_seq_len $CONTEXT --n_ctx $CONTEXT --n-gpu-layers $GPU_LAYERS

if MODEL_TYPE == "gptq":
    LOADER = "EXLLAMAV2_HF"
    !python server.py --share --nowebui --api --public-api $PROVIDER --model $MODEL_NAME --loader $LOADER --max_seq_len $CONTEXT --n_ctx $CONTEXT